In [16]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets


import seaborn as sns

import random

from gustavo_functions import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Cleaning the data 

In [53]:
# Load the datasets (make sure to update the file paths if necessary)
global_df = pd.read_csv("data-project3/GlobalLandTemperaturesByCity.csv")
#green_df = pd.read_csv("data-project3/greenhouse_gas_inventory_data_data(1).csv")
#green_df = pd.read_csv("new_green.csv")
green_df = green
pollution_df = pd.read_csv("data-project3/pollution_us_2000_2016.csv")


### 1 Cleaning the GlobalLandTemperaturesByCity dataset
# Filter only data for the United States
global_df = global_df[global_df["Country"] == "United States"].copy()

# Convert the date column to datetime format
global_df["dt"] = pd.to_datetime(global_df["dt"])

# Extract the year from the date column
global_df["Year"] = global_df["dt"].dt.year

# Remove NaN values in the temperature column
global_df.dropna(subset=["AverageTemperature"], inplace=True)

# Group by city and year, calculating the annual average temperature
global_clean = global_df.groupby(["City", "Year"])['AverageTemperature'].mean().reset_index()

# Extract unique latitude values per city
latitude_df = global_df[["City", "Latitude","Longitude"]].drop_duplicates()

### 2 Cleaning the Greenhouse Gas dataset
# Filter only "United States of America"
#green_df = green_df[green_df["country_or_area"] == "United States of America"].copy()

# Remove NaN values in the value column
green_df.dropna(subset=["value"], inplace=True)

# Keep only key columns
green_clean = green_df[["year", "value"]] #green_df[["year", "value", "category"]]

### 3 Cleaning the Pollution dataset
# Convert the date column to datetime format
pollution_df["Date Local"] = pd.to_datetime(pollution_df["Date Local"])

# Extract the year from the date column
pollution_df["Year"] = pollution_df["Date Local"].dt.year

# Select key columns
pollution_clean = pollution_df[["City", "Year", "NO2 Mean", "SO2 Mean", "CO Mean"]]

# Average pollution values by city and year
pollution_clean = pollution_clean.groupby(["City", "Year"]).mean().reset_index()

###  Merging the datasets
# Merge temperature and pollution data
merged_df = pd.merge(global_clean, pollution_clean, on=["City", "Year"], how="inner")

# Merge with greenhouse gas data
final_df = pd.merge(merged_df, green_clean, left_on="Year", right_on="year", how="inner")

# Drop the duplicate year column
final_df.drop(columns=["year"], inplace=True)

# Merge latitude data
final_df = pd.merge(final_df, latitude_df, on="City", how="left")

# Convert latitude values to numeric format
final_df["Latitude"] = final_df["Latitude"].str.replace("N", "").str.replace("S", "-").astype(float)

# Convert longitude to numeric format
final_df["Longitude"] = final_df["Longitude"].apply(lambda x: float(x[:-1]) * (-1 if x[-1] == 'W' else 1))

###
final_df = final_df.rename(columns={'value': 'CO2-natural-pross'})

# Save the cleaned and merged dataset in CSV format
final_df.to_csv("cleaned_temperature_pollution_data.csv", index=False)

In [41]:
# Set the random seed for reproducibility
np.random.seed(42)

# Load your green dataset
green = pd.read_csv("data-project3/greenhouse_gas_inventory_data_data(1).csv")

# Drop the 'country_or_area' column
green = green.drop(columns=["country_or_area"])

# Load the dataset containing cities 
cities_df = pd.read_csv("data-project3/pollution_us_2000_2016.csv")

# Extract the unique cities from the 'City' column
unique_cities = cities_df["City"].unique()

# Randomly assign a city from the unique cities to each row in the green dataset.
green["cities"] = np.random.choice(unique_cities, size=len(green))

# Save the modified dataset to a new CSV file called 'new_green.csv'
green.to_csv("new_green.csv", index=False)